<a href="https://colab.research.google.com/github/AmeedShaik/BTC_USD_Final/blob/main/BTC_USD_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q streamlit yfinance plotly pandas numpy pyngrok

# --- Save Streamlit BTC App as btc_streamlit.py ---
btc_code = """\
import streamlit as st
import yfinance as yf
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import datetime

st.set_page_config(page_title="BTC-USD Streamlit Viewer", layout="wide")

st.title("BTC-USD — Interactive Viewer")
st.write("Candlesticks + EMA20/50 + RSI(14) + Volume")

# Sidebar
tf = st.sidebar.selectbox("Timeframe", ["15m","1H","4H","1D"], index=1)
if tf in ["15m","1H","4H"]:
    period = st.sidebar.selectbox("Intraday period", ["7d","30d","60d","90d"], index=1)
else:
    start_date = st.sidebar.date_input("Start date", datetime.date(2024,1,1))
    end_date = st.sidebar.date_input("End date", datetime.date.today())

# Data fetch
def fetch_data(tf, period=None, start=None, end=None):
    mapping = {"15m":"15m","1H":"60m","4H":"60m","1D":"1d"}
    interval = mapping[tf]
    if tf=="1D":
        df = yf.download("BTC-USD", start=start, end=end, interval=interval, progress=False)
    else:
        df = yf.download("BTC-USD", period=period, interval=interval, progress=False)
    if tf=="4H" and not df.empty:
        df = df.resample("4H").agg({"Open":"first","High":"max","Low":"min","Close":"last","Volume":"sum"})
    return df

df = pd.DataFrame()
if tf=="1D":
    df = fetch_data(tf, start=start_date, end=end_date)
else:
    df = fetch_data(tf, period=period)

if df.empty:
    st.error("No data fetched")
else:
    df["EMA20"] = df["Close"].ewm(span=20).mean()
    df["EMA50"] = df["Close"].ewm(span=50).mean()
    delta = df["Close"].diff()
    gain = delta.clip(lower=0).rolling(14).mean()
    loss = -delta.clip(upper=0).rolling(14).mean()
    rs = gain / loss
    df["RSI"] = 100 - (100/(1+rs))

    rows = 3
    fig = make_subplots(rows=rows, cols=1, shared_xaxes=True,
                        row_heights=[0.6,0.2,0.2], vertical_spacing=0.02,
                        specs=[[{"type":"candlestick"}],[{"type":"bar"}],[{"type":"scatter"}]])

    fig.add_trace(go.Candlestick(x=df.index, open=df["Open"], high=df["High"],
                                 low=df["Low"], close=df["Close"], name="Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df["EMA20"], name="EMA20"), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df["EMA50"], name="EMA50"), row=1, col=1)

    fig.add_trace(go.Bar(x=df.index, y=df["Volume"], name="Volume"), row=2, col=1)

    fig.add_trace(go.Scatter(x=df.index, y=df["RSI"], name="RSI"), row=3, col=1)
    fig.add_hline(y=70, line_dash="dash", row=3, col=1)
    fig.add_hline(y=30, line_dash="dash", row=3, col=1)

    fig.update_layout(height=800, title=f"BTC-USD {tf}")
    st.plotly_chart(fig, use_container_width=True)
"""

with open("btc_streamlit.py", "w") as f:
    f.write(btc_code)

# --- Run Streamlit with ngrok to get public URL ---
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print("Click this link to open the Streamlit app:", public_url)
!streamlit run btc_streamlit.py --server.port 8501


Click this link to open the Streamlit app: NgrokTunnel: "https://128689352885.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.148.201.168:8501

/content/btc_streamlit.py:28: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download("BTC-USD", period=period, interval=interval, progress=False)
/content/btc_streamlit.py:28: FutureWarning:

YF.download() has changed argument auto_adjust default to True

